# IBM Data Science Specialization Capstone Project Notebook

### Importing libraries

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Get HTML data from wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
PostalCode = []
Borough = []
Neighborhood = []

for data in soup.tbody.find_all('tr')[1:]:
    PostalCode.append(data.find('td').text)
    Borough.append(data.find_all('td')[1].text)
    Neighborhood.append(data.find_all('td')[2].text[:-1])

In [5]:
data = {'PostalCode': PostalCode,
        'Borough': Borough,
        'Neighborhood': Neighborhood}
df = pd.DataFrame(data)
# Drop rows where Borough is 'Not assigned', then reset index
df = df[df.Borough != 'Not assigned'].reset_index().drop('index', axis = 1)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [6]:
df["PostalCode"][10]

'M3B'

cummulating the data separated with commas

In [7]:
i = 1
while(i < len(df)):
    if df['PostalCode'].iloc[i] == df['PostalCode'].iloc[i - 1]:
        df.at[i - 1, 'Neighborhood'] = df.Neighborhood.iloc[i - 1] +', ' + df.Neighborhood.iloc[i]
        df.drop(index = i, inplace = True)
        df = df.reset_index().drop('index', axis = 1)
    else:
        i += 1

In [8]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Removing not assigned

In [9]:
for i in range(0,len(df)):
    if df["Neighborhood"][i]=="Not assigned":
        df["Neighborhood"][i]=df['Borough'][i]

In [10]:
df.shape

(103, 3)

### extracting longitudes and latitudes

In [11]:
Geo = pd.read_csv("Geospatial_Coordinates.csv") 
Geo.rename({'Postal Code': 'PostalCode'}, axis='columns', inplace=True)

In [12]:
df = pd.merge(df, Geo, on='PostalCode')
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### separating Toronto

In [13]:
df1=df

i=0
while i<len(df1):
    if str(df1['Borough'][i]).find("Toronto")!=-1:
            i+=1
    else:
        df1.drop(index = i, inplace = True)
        df1 = df1.reset_index().drop('index', axis = 1)   



In [14]:
df1.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [15]:
#!pip3 install geopy
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Mapping Data

In [16]:
#!pip3 install folium
import folium
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [17]:
CLIENT_ID = 'IAMYLYPSDNNC5YZQFT3ANBIS1YXBMCOOKH105EXS3XB2WV25' # your Foursquare ID
CLIENT_SECRET = 'SNS53E13OE0OJHKJ3VXIZUFVKYRQHSCBMTVGVIX52XFNRKUS' # your Foursquare Secret
VERSION = '20190701' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IAMYLYPSDNNC5YZQFT3ANBIS1YXBMCOOKH105EXS3XB2WV25
CLIENT_SECRET:SNS53E13OE0OJHKJ3VXIZUFVKYRQHSCBMTVGVIX52XFNRKUS


### Get the neighborhood's latitude and longitude values.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("Finished!")
    
    return(nearby_venues)

In [19]:
Toronto_venues = getNearbyVenues(names=df1['Neighborhood'],
                                 latitudes=df1['Latitude'],
                                 longitudes=df1['Longitude'],
                                 radius = 1000
                                )

Finished!


In [20]:
print(Toronto_venues.shape)
Toronto_venues.head()

(3093, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Harbourfront, Regent Park",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [21]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,48,48,48,48,48,48
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",38,38,38,38,38,38
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


In [22]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


### Explore Neighborhoods in Toronto

In [23]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.00,0.00,0.00,0.0000,0.0000,0.040000,0.00,0.00,...,0.01,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.010000
1,Berczy Park,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.00,0.01,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.00,0.00,0.00,0.0000,0.0000,0.020833,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.00,0.00,0.00,0.0625,0.0625,0.000000,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
5,"Cabbagetown, St. James Town",0.00,0.00,0.00,0.00,0.0000,0.0000,0.026316,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
6,Central Bay Street,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.01,0.030000,0.00,0.00,0.000000,0.00,0.010000,0.00,0.0000,0.010000
7,"Chinatown, Grange Park, Kensington Market",0.00,0.00,0.00,0.00,0.0000,0.0000,0.000000,0.00,0.00,...,0.00,0.080000,0.00,0.00,0.030000,0.00,0.010000,0.00,0.0000,0.020000
8,Christie,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.00,0.0000,0.000000
9,Church and Wellesley,0.00,0.00,0.01,0.01,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.010000,0.01,0.01,0.010000,0.00,0.000000,0.01,0.0000,0.010000


### Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.06
1          Coffee Shop  0.05
2                Hotel  0.05
3  American Restaurant  0.04
4              Theater  0.04


----Berczy Park----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2        Hotel  0.06
3   Restaurant  0.04
4     Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.06
2                     Bar  0.05
3  Furniture / Home Store  0.04
4              Restaurant  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0                Park  0.10
1         Coffee Shop  0.06
2             Brewery  0.06
3    Sushi Restaurant  0.04
4  Italian Restaurant  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0       Coffee Shop  0.19
1

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,American Restaurant,Theater,Steakhouse,Sushi Restaurant,Bakery,Japanese Restaurant,Bar
1,Berczy Park,Coffee Shop,Hotel,Café,Beer Bar,Restaurant,Japanese Restaurant,Bakery,Park,Art Gallery,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bar,Furniture / Home Store,Bakery,Restaurant,Tibetan Restaurant,Caribbean Restaurant,Arts & Crafts Store,Beer Bar
3,Business Reply Mail Processing Centre 969 Eastern,Park,Coffee Shop,Brewery,Burrito Place,Pizza Place,Pet Store,Sushi Restaurant,Italian Restaurant,Restaurant,Steakhouse
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Harbor / Marina,Café,Track,Garden,Sculpture Garden,Airport,Airport Lounge,Dog Run,Scenic Lookout


### Cluster Neighborhoods

In [28]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 2, 1, 1, 1, 1, 0], dtype=int32)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Theater,Italian Restaurant,Restaurant,Pub,Bakery,Diner,Breakfast Spot,Park
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Middle Eastern Restaurant,Café,Restaurant,Italian Restaurant,Gastropub,Tea Room,Clothing Store,Gym,Sushi Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Gastropub,Seafood Restaurant,Cosmetics Shop,Italian Restaurant,Breakfast Spot
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Park,Bar,Bakery,Tea Room
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Hotel,Café,Beer Bar,Restaurant,Japanese Restaurant,Bakery,Park,Art Gallery,Farmers Market


In [31]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Theater,Italian Restaurant,Restaurant,Pub,Bakery,Diner,Breakfast Spot,Park
3,East Toronto,0,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Park,Bar,Bakery,Tea Room
8,West Toronto,0,Café,Coffee Shop,Park,Italian Restaurant,Bar,Brewery,Bakery,Gourmet Shop,Portuguese Restaurant,Pharmacy
11,East Toronto,0,Greek Restaurant,Coffee Shop,Café,Pub,Fast Food Restaurant,Pizza Place,Italian Restaurant,Ice Cream Shop,Diner,Spa
18,Central Toronto,0,Sushi Restaurant,Asian Restaurant,Café,Bank,Pharmacy,Italian Restaurant,Coffee Shop,Bagel Shop,Japanese Restaurant,Deli / Bodega
19,Central Toronto,0,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Café,Pizza Place,Pharmacy,Sushi Restaurant,Dessert Shop,Gym,Diner
20,Central Toronto,0,Park,Italian Restaurant,Coffee Shop,Café,Bank,Burger Joint,Japanese Restaurant,Sushi Restaurant,Trail,Juice Bar
22,Central Toronto,0,Coffee Shop,Italian Restaurant,Sporting Goods Shop,Park,Café,Diner,Mexican Restaurant,Skating Rink,Sushi Restaurant,Flower Shop
24,West Toronto,0,Bar,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Breakfast Spot,Bakery,Thai Restaurant,Eastern European Restaurant,Park
25,Central Toronto,0,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Café,Pub,Gym,Fast Food Restaurant,Indian Restaurant,Dessert Shop


#### Cluster 2

In [33]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Coffee Shop,Middle Eastern Restaurant,Café,Restaurant,Italian Restaurant,Gastropub,Tea Room,Clothing Store,Gym,Sushi Restaurant
2,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Gastropub,Seafood Restaurant,Cosmetics Shop,Italian Restaurant,Breakfast Spot
4,Downtown Toronto,1,Coffee Shop,Hotel,Café,Beer Bar,Restaurant,Japanese Restaurant,Bakery,Park,Art Gallery,Farmers Market
5,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Ramen Restaurant,Japanese Restaurant,Arts & Crafts Store,Vegetarian / Vegan Restaurant,Bubble Tea Shop,Park,Gym
6,Downtown Toronto,1,Korean Restaurant,Café,Coffee Shop,Grocery Store,Indian Restaurant,Ice Cream Shop,Cocktail Bar,Bar,Ethiopian Restaurant,Pizza Place
7,Downtown Toronto,1,Café,Coffee Shop,Hotel,American Restaurant,Theater,Steakhouse,Sushi Restaurant,Bakery,Japanese Restaurant,Bar
9,Downtown Toronto,1,Coffee Shop,Hotel,Café,Italian Restaurant,Brewery,Bakery,Scenic Lookout,Park,Aquarium,Theater
10,West Toronto,1,Café,Bar,Bakery,Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Pizza Place
12,Downtown Toronto,1,Café,Coffee Shop,Hotel,Italian Restaurant,Steakhouse,Concert Hall,American Restaurant,Bakery,Beer Bar,Restaurant
13,West Toronto,1,Café,Coffee Shop,Bar,Furniture / Home Store,Bakery,Restaurant,Tibetan Restaurant,Caribbean Restaurant,Arts & Crafts Store,Beer Bar


#### Cluster 3

In [34]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Downtown Toronto,2,Coffee Shop,Harbor / Marina,Café,Track,Garden,Sculpture Garden,Airport,Airport Lounge,Dog Run,Scenic Lookout


#### Cluster 4

In [35]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Bookstore,College Quad,Café,College Gym,Coffee Shop,Gym / Fitness Center,Park,Trail,Dumpling Restaurant,Dog Run


#### Cluster 5

In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downtown Toronto,4,Coffee Shop,Bank,Grocery Store,Park,Breakfast Spot,Smoothie Shop,Japanese Restaurant,Filipino Restaurant,Metro Station,Building
